# 读取数据

In [2]:
import pymongo
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
client = pymongo.MongoClient('localhost',27017)
db  = client['Ziru_housing']
table = db['Shanghai_Renting']
data = pd.DataFrame(list(table.find()))
del data['_id']

In [3]:
data.head()

,area,district,metro,metro_distance,price,square,title,type
0,42 ㎡06/6层2室1厅,徐汇,距3号线龙漕路站173米,173,4830,42,整租 · 上缝小区2居室-南,整租
1,68.65 ㎡06/6层2室1厅,徐汇,距11号线龙耀路站757米,757,5130,68.65,整租 · 滨江小区2居室-南,整租
2,30.37 ㎡06/6层1室1厅,徐汇,距4号线上海体育馆站236米,236,4690,30.37,整租 · 中原新村1居室-南,整租
3,49.15 ㎡01/6层2室1厅,徐汇,距3号线石龙路站243米,243,4790,49.15,整租 · 东泉小区2居室-南,整租
4,33.17 ㎡04/6层1室1厅,徐汇,距4号线东安路站488米,488,4490,33.17,整租 · 斜土路2200弄1居室-南,整租


In [4]:
data.shape
#data[data['square']=='约13']

(14652, 8)

---

---

# 正则提取float

In [5]:
import re
def square(num):
    # 取薪资均值----------------
    pattern = re.compile('[1-9]\d*.\d*|0.\d*[1-9]\d*')
    temp = num
    res = re.findall(pattern, temp)
    result = 0
    for i in res:
        result = float(i)
    # 函数返回值
    return result

square_list = []
metro_list = []
for i in range(0,14652):
    square_num = square(data['square'][i])
    metro_num =  square(data['metro_distance'][i])
    square_list.append(square_num)
    metro_list.append(metro_num)
#print(square_list)

In [6]:
sq = Series(square_list)
md = Series(metro_list)
data.insert(6,'S',sq)
data.insert(4,'metro_num',md)

In [7]:
data.head()

,area,district,metro,metro_distance,metro_num,price,square,S,title,type
0,42 ㎡06/6层2室1厅,徐汇,距3号线龙漕路站173米,173,173.0,4830,42,42.00,整租 · 上缝小区2居室-南,整租
1,68.65 ㎡06/6层2室1厅,徐汇,距11号线龙耀路站757米,757,757.0,5130,68.65,68.65,整租 · 滨江小区2居室-南,整租
2,30.37 ㎡06/6层1室1厅,徐汇,距4号线上海体育馆站236米,236,236.0,4690,30.37,30.37,整租 · 中原新村1居室-南,整租
3,49.15 ㎡01/6层2室1厅,徐汇,距3号线石龙路站243米,243,243.0,4790,49.15,49.15,整租 · 东泉小区2居室-南,整租
4,33.17 ㎡04/6层1室1厅,徐汇,距4号线东安路站488米,488,488.0,4490,33.17,33.17,整租 · 斜土路2200弄1居室-南,整租


---

# 去重

In [8]:
data = data.drop_duplicates()
data.shape

(14453, 10)

---

# 替换值

In [9]:
data['metro_num'] = data['metro_num'].replace(0,np.nan)
data['price'] = data['price'].replace(0,np.nan)
data['square'] = data['square'].replace(0,np.nan)

In [10]:
#data[data['metro_num']==np.nan]
data['price'] = pd.to_numeric(data['price'])

## 将按天付款的房源价格乘以30

In [11]:
data['mon_price'] = data['price'] 
data.loc[data['price']<600,'mon_price'] *= 30        # 关键句

In [12]:
data = data.dropna()

In [13]:
data.shape

(14034, 11)

---

# 输出到csv

In [14]:
data.to_csv('housing_shanghai.csv')

---

# 准备数据

In [15]:
housing = data.copy()

In [16]:
housing.shape

(14034, 11)

In [17]:
housing['mon_price'] = pd.to_numeric(housing['mon_price'])

In [18]:
housing = housing[['district','type','metro_num','S','mon_price']]
housing.head()

,district,type,metro_num,S,mon_price
0,徐汇,整租,173.0,42.00,4830
1,徐汇,整租,757.0,68.65,5130
2,徐汇,整租,236.0,30.37,4690
3,徐汇,整租,243.0,49.15,4790
4,徐汇,整租,488.0,33.17,4490


In [19]:
housing_num = housing[['metro_num','S','mon_price']]
housing_num.head()

,metro_num,S,mon_price
0,173.0,42.00,4830
1,757.0,68.65,5130
2,236.0,30.37,4690
3,243.0,49.15,4790
4,488.0,33.17,4490


In [20]:
housing_cat = housing[['district','type']]
housing_cat.head()

,district,type
0,徐汇,整租
1,徐汇,整租
2,徐汇,整租
3,徐汇,整租
4,徐汇,整租


---

# 流水线

## 特征缩放

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer

num_pipeline = Pipeline([
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler())
    ])

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [22]:
ss=StandardScaler()
housing_num_tr = ss.fit_transform(housing_num)
ss.inverse_transform(housing_num_tr,copy=True)[:5]

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\13626\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[ 173.  ,   42.  , 4830.  ],
       [ 757.  ,   68.65, 5130.  ],
       [ 236.  ,   30.37, 4690.  ],
       [ 243.  ,   49.15, 4790.  ],
       [ 488.  ,   33.17, 4490.  ]])

## 数值和类别的流水线

In [23]:
from sklearn.compose import ColumnTransformer

In [24]:
num_attribs = list(housing_num)
cat_attribs = ['district','type']

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing).toarray()

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [25]:
housing_prepared

array([[-1.26863157,  1.14766383,  1.46682065, ...,  0.        ,
         0.        ,  1.        ],
       [-0.11263402,  2.55204439,  1.67270329, ...,  0.        ,
         0.        ,  1.        ],
       [-1.14392635,  0.53479532,  1.37074209, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.50061597, -0.5175679 , -0.64004503, ...,  0.        ,
         1.        ,  0.        ],
       [-0.3224555 , -1.06561885, -0.61945677, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.00811229, -0.6176926 , -0.77730012, ...,  0.        ,
         1.        ,  0.        ]])

---

# 聚类

In [26]:
from sklearn.cluster import KMeans # 导入Kmeans
k=5 # 聚类类别数
kmodel = KMeans(n_clusters=k, n_jobs=4)
kmodel.fit(housing_prepared)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=4, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## 聚类结果的前三列

In [27]:
kmodel.cluster_centers_[0:5,:3]

array([[ 1.67691161, -0.37531457, -0.61829816],
       [-0.82639305, -0.40914955, -0.11939841],
       [-0.25671001,  3.22776121,  2.87673547],
       [-0.32262087,  1.37090281,  1.27213038],
       [ 0.17774179, -0.42666866, -0.51390364]])

## 聚类结果反特征缩放

In [28]:
ss.inverse_transform(kmodel.cluster_centers_[0:5,:3],copy=True)

array([[1661.06303972,   13.09944732, 1791.68825561],
       [ 396.4150863 ,   12.45738334, 2518.65544428],
       [ 684.21405229,   81.47263072, 6884.44444444],
       [ 650.9164557 ,   46.23625823, 4546.30886076],
       [ 903.69535827,   12.12493466, 1943.80576836]])

分别为：距离地铁的距离，面积，价格

---

## 将聚类类别插入DataFrame

In [29]:
housing_data = housing.copy()
housing_data.head(1)

,district,type,metro_num,S,mon_price
0,徐汇,整租,173.0,42.0,4830


In [30]:
label = Series(kmodel.labels_,index=housing_data.index)

housing_data.insert(5,'聚类类别',label)
#kmeans_job

In [31]:
housing_data.head()

,district,type,metro_num,S,mon_price,聚类类别
0,徐汇,整租,173.0,42.00,4830,3
1,徐汇,整租,757.0,68.65,5130,3
2,徐汇,整租,236.0,30.37,4690,3
3,徐汇,整租,243.0,49.15,4790,3
4,徐汇,整租,488.0,33.17,4490,3


---

## 转换为房屋等级

In [32]:
c = pd.DataFrame(kmodel.cluster_centers_).sort_values(2)
c['房屋等级']=['交通不便普通','交通一般普通','交通便利普通','高端房源','豪宅']
#c

In [33]:
labels = c
# 高价值
expensive = labels[labels['房屋等级']=='豪宅'].index[0]
high = labels[labels['房屋等级']=='高端房源'].index[0]
mid = labels[labels['房屋等级']=='交通便利普通'].index[0]
normal = labels[labels['房屋等级']=='交通一般普通'].index[0]
low = labels[labels['房屋等级']=='交通不便普通'].index[0]
#print(expensive,high,mid,low)

In [34]:
kmeans_type_shift = {
    expensive:'豪宅',
    high:'高端房源',
    mid:'交通便利普通',
    normal:'交通一般普通',
    low:'交通不便普通'
}
housing_data['聚类类别'] = housing_data['聚类类别'].map(kmeans_type_shift)

In [35]:
housing_data.head()

,district,type,metro_num,S,mon_price,聚类类别
0,徐汇,整租,173.0,42.00,4830,高端房源
1,徐汇,整租,757.0,68.65,5130,高端房源
2,徐汇,整租,236.0,30.37,4690,高端房源
3,徐汇,整租,243.0,49.15,4790,高端房源
4,徐汇,整租,488.0,33.17,4490,高端房源


---

In [36]:
housing_data.shape
#housing_data.info()

(14034, 6)

In [95]:
housing_data.to_csv('housing_with_type.csv')

---

# 聚类后的Groupby

In [37]:
grouped_house = housing_data[['metro_num','S','mon_price']].groupby(housing_data['聚类类别'])
#grouped_house

In [38]:
grouped_house.mean().sort_values(by='mon_price',ascending=True).round(2)

,metro_num,S,mon_price
聚类类别,,,
交通不便普通,1661.06,13.10,1791.69
交通一般普通,903.70,12.12,1943.81
交通便利普通,396.42,12.46,2518.66
高端房源,650.92,46.24,4546.31
豪宅,684.21,81.47,6884.44


---

# 分区

In [39]:
housing_data['district'].value_counts() # 各区房源总数

浦东    2220
宝山    1925
闵行    1883
嘉定    1569
松江    1565
普陀    1372
徐汇     864
杨浦     645
长宁     555
闸北     484
虹口     446
黄浦     270
青浦     182
静安      54
Name: district, dtype: int64

---

# 转化为全上海类别的DataFrame

In [60]:
low_type = housing_data[housing_data['聚类类别']=='交通不便普通']['district'].value_counts()
normal_type = housing_data[housing_data['聚类类别']=='交通一般普通']['district'].value_counts()
mid_type = housing_data[housing_data['聚类类别']=='交通便利普通']['district'].value_counts()
high_type = housing_data[housing_data['聚类类别']=='高端房源']['district'].value_counts()
expensive_type = housing_data[housing_data['聚类类别']=='豪宅']['district'].value_counts()

In [61]:
low_type = Series(low_type)
normal_type = Series(normal_type)
mid_type = Series(mid_type)
high_type = Series(high_type)
expensive_type = Series(expensive_type)

In [62]:
temp_pd = DataFrame(housing_data['district'].value_counts())

In [63]:
# '交通不便普通','交通一般普通','交通便利普通','高端房源','豪宅'
temp_pd.insert(1,'交通不便普通',low_type)
temp_pd.insert(2,'交通一般普通',normal_type)
temp_pd.insert(3,'交通便利普通',mid_type)
temp_pd.insert(4,'高端房源',high_type)
temp_pd.insert(5,'豪宅',expensive_type)

In [64]:
shanghai_housing = temp_pd.fillna(0).astype(int)

In [65]:
shanghai_housing

,district,交通不便普通,交通一般普通,交通便利普通,高端房源,豪宅
浦东,2220,164,774,765,416,101
宝山,1925,464,897,371,140,53
闵行,1883,282,848,518,185,50
嘉定,1569,624,479,242,167,57
松江,1565,405,641,403,82,34
普陀,1372,204,290,613,213,52
徐汇,864,32,42,541,180,69
杨浦,645,24,162,264,163,32
长宁,555,14,72,306,115,48
闸北,484,32,119,206,88,39


In [66]:
shanghai_housing.to_csv('shanghai_housing_type.csv')

---

## 占比分析

In [83]:
housing_rate = shanghai_housing.copy()

In [84]:
housing_rate.insert(6,'豪宅占比',(shanghai_housing['豪宅']/shanghai_housing['district']).round(3))
housing_rate.insert(7,'高端房源占比',(shanghai_housing['高端房源']/shanghai_housing['district']).round(3))
housing_rate.insert(8,'交通便利普通房源占比',(shanghai_housing['交通便利普通']/shanghai_housing['district']).round(3))
housing_rate.insert(9,'交通一般普通房源占比',(shanghai_housing['交通一般普通']/shanghai_housing['district']).round(3))
housing_rate.insert(10,'交通不便普通房源占比',(shanghai_housing['交通不便普通']/shanghai_housing['district']).round(3))
housing_rate

,district,交通不便普通,交通一般普通,交通便利普通,高端房源,豪宅,豪宅占比,高端房源占比,交通便利普通房源占比,交通一般普通房源占比,交通不便普通房源占比
浦东,2220,164,774,765,416,101,0.045,0.187,0.345,0.349,0.074
宝山,1925,464,897,371,140,53,0.028,0.073,0.193,0.466,0.241
闵行,1883,282,848,518,185,50,0.027,0.098,0.275,0.450,0.150
嘉定,1569,624,479,242,167,57,0.036,0.106,0.154,0.305,0.398
松江,1565,405,641,403,82,34,0.022,0.052,0.258,0.410,0.259
普陀,1372,204,290,613,213,52,0.038,0.155,0.447,0.211,0.149
徐汇,864,32,42,541,180,69,0.080,0.208,0.626,0.049,0.037
杨浦,645,24,162,264,163,32,0.050,0.253,0.409,0.251,0.037
长宁,555,14,72,306,115,48,0.086,0.207,0.551,0.130,0.025
闸北,484,32,119,206,88,39,0.081,0.182,0.426,0.246,0.066


In [94]:
from pyecharts import Bar

bar = Bar("上海各类房源")

districts = housing_rate.index
y1 = housing_rate['豪宅占比'].values
y2 = housing_rate['高端房源占比'].values
y3 = housing_rate['交通便利普通房源占比'].values
y4 = housing_rate['交通一般普通房源占比'].values
y5 = housing_rate['交通不便普通房源占比'].values

bar.add("豪宅", districts, y1,xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30)
bar.add("高端房源", districts, y2, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30)
bar.add("交通便利普通房源", districts, y3, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30)
bar.add("交通一般普通房源", districts, y4, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30)
bar.add("交通不便普通房源", districts, y5, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30)

bar

In [92]:
v1 = [5, 20, 36, 10, 75, 90]
v2 = [10, 25, 8, 60, 20, 80]
bar = Bar("柱状图数据堆叠示例",title_pos='center')
bar.add("豪宅", districts, y1,xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30,
        is_stack=True,legend_pos="left",legend_orient="vertical")
bar.add("高端房源", districts, y2, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30,
        is_stack=True,legend_pos="left",legend_orient="vertical")
bar.add("交通便利普通房源", districts, y3, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30,
        is_stack=True,legend_pos="left",legend_orient="vertical")
bar.add("交通一般普通房源", districts, y4, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30,
        is_stack=True,legend_pos="left",legend_orient="vertical")
bar.add("交通不便普通房源", districts, y5, xaxis_interval=0, xaxis_rotate=30, yaxis_rotate=30,
        is_stack=True,legend_pos="left",legend_orient="vertical")

bar

### 图表中获取信息

- ***豪宅与高端房源***堆叠占比显示出的数据，**静安区**与**黄浦区**远高于上海其他各区；

- 根据***交通不便的普通房源***与***交通一般的普通房源***堆叠图显示，**静安区**、**黄浦区**与**徐汇区**的占比极低，由此可以分析出上海市最繁华的商圈/行政区。

- 根据***交通一般的普通房源***与***交通不便的普通房源***堆叠图显示，**宝山**、**闵行**、**松江**、**嘉定**的房源占比较多，因此如果不介意通勤麻烦可以选择这四个商圈/行政区作为租房首选。

- ***交通便利的普通房源***图表中，**徐汇区**、**长宁区**、**黄浦区**、**青浦区**的房源占比较多，因此往往可以选择这四个商圈/行政区作为租房首选。

---

# 可视化

In [115]:
housing_data[housing_data['district']=='浦东']['聚类类别'].value_counts() # 各区房源总数

交通一般普通    774
交通便利普通    765
高端房源      416
交通不便普通    164
豪宅        101
Name: 聚类类别, dtype: int64

In [116]:
from pyecharts import Pie

x = ["交通不便普通", "交通一般普通", "交通便利普通", "高端房源", "豪宅"]
y = [765, 774, 164, 416, 101]
pie = Pie("浦东不同类别房源",title_pos='center')
pie.add("", x, y, is_label_show=True,legend_pos="left",legend_orient="vertical")
pie

---

## 3D bar chart

In [110]:
from pyecharts import Bar3D

bar3d = Bar3D("", width=1200, height=600)
x_axis = ["交通不便普通", "交通一般普通", "交通便利普通", "高端房源", "豪宅"]
y_aixs = housing_rate.index
data = [[0,0,164],[1,0,774],[2,0,765],[3,0,416],[4,0,101],
        [0,1,464],[1,1,897],[2,1,371],[3,1,140],[4,1,53],
        [0,2,282],[1,2,848],[2,2,518],[3,2,185],[4,2,50],
        [0,3,624],[1,3,479],[2,3,242],[3,3,167],[4,3,57],
        [0,4,405],[1,4,641],[2,4,403],[3,4,82],[4,4,34],
        [0,5,204],[1,5,290],[2,5,613],[3,5,213],[4,5,52],
        [0,6,32],[1,6,42],[2,6,54],[3,6,180],[4,6,69],
        [0,7,24],[1,7,162],[2,7,264],[3,7,163],[4,7,32],
        [0,8,14],[1,8,72],[2,8,306],[3,8,115],[4,8,48],
        [0,9,32],[1,9,119],[2,9,206],[3,9,88],[4,9,39],
        [0,10,39],[1,10,71],[2,10,212],[3,10,105],[4,10,19],
        [0,11,0],[1,11,1],[2,11,144],[3,11,86],[4,11,39],
        [0,12,32],[1,12,42],[2,12,90],[3,12,11],[4,12,7],
        [0,13,0],[1,13,0],[2,13,18],[3,13,24],[4,13,12]
       ]
# 0,0,18,24,12
range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
bar3d.add("", x_axis, y_aixs, [[d[0], d[1], d[2]] for d in data], 
          is_visualmap=True,visual_range=[0, 900], visual_range_color=range_color, 
          grid3d_width=100, grid3d_depth=150,
         yaxis3d_interval=0, xaxis_rotate=30, yaxis_rotate=30,
          is_grid3d_rotate=True,)

bar3d